In [ ]:
import os

import joblib
from optuna import create_study
from optuna import Trial
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
import pandas as pd
from sklearn import metrics
from sklearn import feature_extraction
from sklearn import preprocessing
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier, plot_importance # <- extreme gradient boosting model

from src.utils import plot_metric_curves


# Ajuste de Hiperparametros

In [ ]:
LABEL_COLUMN = "churned"
CATEGORICAL_COLUMNS = [
    "country_name",
    "device_os",
    "device_lang",
]
NUMERICAL_COLUMNS = [
    "cnt_user_engagement",
    "cnt_level_start_quickplay",
    "cnt_level_end_quickplay",
    "cnt_level_complete_quickplay",
    "cnt_level_reset_quickplay",
    "cnt_post_score",
    "cnt_spend_virtual_currency",
    "cnt_ad_reward",
    "cnt_challenge_a_friend",
    "cnt_completed_5_levels",
    "cnt_use_extra_steps",
]
IGNORE_COLUMNS = [
    "user_first_engagement",
    "user_pseudo_id",
    "is_enable",
    "bounced",
    "device_lang",
]

transformer_numeric = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
        ("scaler", preprocessing.StandardScaler()),
    ]
)
transformer_categorical = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("one-hot", preprocessing.OneHotEncoder(handle_unknown="ignore")),
    ]
)
transformer_pipeline = ColumnTransformer(
    transformers=[
        ("numeric_features", transformer_numeric, NUMERICAL_COLUMNS),
        ("categorical_features", transformer_categorical, CATEGORICAL_COLUMNS),
        ("ignore_features", "drop", IGNORE_COLUMNS),
    ]
)

In [ ]:
def objective(trial):
    train = pd.read_csv("../data/users_train.csv")

    x_train, y_train = train.drop(columns=[LABEL_COLUMN]), train[LABEL_COLUMN]
    scale_pos_weight = 1 / y_train.mean()

    folds = 10
    shuffle = True
    seed = 42
    kf = KFold(n_splits=folds, shuffle=shuffle, random_state=seed)

    param = {
        "scale_pos_weight": scale_pos_weight,
        "use_label_encoder": False,
        "eval_metric": "logloss",
        "verbosity": 0,
        "objective": "binary:logistic",
        "tree_method": "exact",  # use exact for small dataset.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        param["max_depth"] = trial.suggest_int("max_depth", 3, 7, step=1)
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical(
            "grow_policy", ["depthwise", "lossguide"]
        )

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical(
            "sample_type", ["uniform", "weighted"]
        )
        param["normalize_type"] = trial.suggest_categorical(
            "normalize_type", ["tree", "forest"]
        )
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    score = []
    for train_idx, valid_idx in kf.split(x_train, y_train):
        x_train_data = x_train.iloc[train_idx,:]
        y_train_data = y_train[train_idx]
        x_valid_data = x_train.iloc[valid_idx,:]
        y_valid_data = y_train[valid_idx]
        xgb_model = Pipeline(
            steps=[
                ("preprocessor", transformer_pipeline),
                (
                    "classifier",
                    XGBClassifier(**param),
                ),
            ]
        )
        xgb_model.fit(x_train_data, y_train_data)

        y_pred = xgb_model.predict(x_valid_data)
        val_score = metrics.roc_auc_score(y_valid_data, y_pred)
        
        score.append(val_score)

    return sum(score)/folds

In [ ]:
study = create_study(direction="maximize")
study.optimize(objective, n_trials=20, timeout=600)

In [ ]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")

trial.params

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_parallel_coordinate(study)

## El mejor modelo

In [ ]:
train = pd.read_csv('../data/users_train.csv')
test = pd.read_csv('../data/users_test.csv')

x_train = train.drop(columns=[LABEL_COLUMN])
y_train = train[LABEL_COLUMN]
x_test = test.drop(columns=[LABEL_COLUMN])
y_test = test[LABEL_COLUMN]

scale_pos_weight = 1/y_train.mean()

In [ ]:
best_parameters = study.best_trial.params
best_parameters = study.trials[1].params
xgb_model_full = Pipeline(
    steps=[
        ("preprocessor", transformer_pipeline),
        (
            "classifier",
            XGBClassifier(
                scale_pos_weight=scale_pos_weight,
                use_label_encoder=False,
                eval_metric='logloss',
                **best_parameters,
            ),
        ),
    ]
)
xgb_model_full.fit(x_train, y_train)

In [ ]:
x = x_test
y = y_test

y_pred = xgb_model_full.predict(x)
display(print(metrics.classification_report(y, y_pred)))
ax = plot_metric_curves(xgb_model_full, x, y)

In [ ]:
y_pred_prob = xgb_model_full.predict_proba(x)

In [ ]:
pd.Series(y_pred_prob[:, 0]).hist(bins=100, range=(0,1))